# AgriWeather
- Data Source : [농촌진흥청 국립농업과학원_농업기상 기본 관측데이터 조회](https://www.data.go.kr/data/15078057/openapi.do)

- Data Format : XML

- 농업기상 관측데이터 (기온, 습도, 풍향, 풍속, 강수량, 일조시간, 일사량 등)을 정보를 제공합니다.

- 관측지점명, 관측지점코드, 조회일자, 조회월, 조회년, 조회기간을 조건으로 조회할 수 있습니다. (10분, 시간, 일, 월, 순, 반순)


---
## 요청변수(Request Parameter)
|항목명(국문)|항목명(영문)|항목크기|항목구분|샘플데이터|항목설명|
|-|-|-|-|-|-|
|인증키|serviceKey|100|필|인증키|공공데이터포털에서 발급받은 인증키|
|페이지 번호|Page_No|10|필|1|페이지 번호|
|한 페이지 결과 수|Page_Size|10|필|20|한 페이지 결과 수 (1 ~ 100 최대 허용)|
|관측년도|search_Year|4|필|2018|관측년도|
|관측지점코드|obsr_Spot_Code|10|필|210852A001|관측지점코드|


## 출력결과(Response Element)
|항목명(국문)|항목명(영문)|항목크기|항목구분|샘플데이터|항목설명|
|-|-|-|-|-|-|
|결과코드|result_Code|3|필|200|결과코드|
|결과메세지|result_Msg|50|필|OK|결과메시지|
|한 페이지 결과 수|rcdcnt|4|필|20|한 페이지 결과 수|
|페이지 번호|page_No|4|필|1|페이지 번호|
|전체 결과 수|total_Count|4|필|29117|전체 결과 수|
|번호|no|10|필|1|데이터 검색결과 정렬 번호|
|관측지점코드|stn_Code|10|필|477802A001|관측지점코드|
|관측지점명|stn_Name|30|필|가평군 가평읍|관측지점명|
|관측시각|date|16|필|2018-01-01 00:10|관측시각|
|기온|temp|12|필|2.1|기온(℃)|
|최고기온|max_Temp|12|필|-1.9|최고기온(℃)|
|최저기온|min_Temp|12|필|-2.5|최저기온(℃)|
|습도|hum|12|필|41.6|습도(%)|
|풍향|widdir|12|필|172.8|풍향|
|풍속|wind|12|필|1|풍속(m/s)|
|강수량|rain|12|필|0|강수량(mm)|
|일조시간|sun_Time|12|필|227|일조시간(MM)|
|일사량|sun_Qy|12|필|0|일사량(MJ/m²)|
|결로시간|condens_Time|12|필|0|결로시간(MM)|
|초상온도|gr_Temp|12|필|-3.8|초상온도(℃)|
|지중온도|soil_Temp|12|필|-0.6|지중온도(℃)|
|토양수분보정값|soil_Wt|12|필|13|토양수분보정값(%)|



In [60]:
import json
import os
import sys
import glob
from io import BytesIO
from zipfile import ZipFile
import time
from pprint import pprint

import numpy as np
import pandas as pd
import requests

from bs4 import BeautifulSoup

import xml.etree.ElementTree as et
import xmltodict


sys.path.append("../import")
import auth

In [43]:
authkey = r"""b8m/px7AI1NzkWN/lcRj9uXgO56otbz7ZIBfXcplZtC7kmPv1qdNbWD9koAjSQp6t+eo2JJPcn+FiGNuklo98A=="""
url = "http://apis.data.go.kr/1390802/AgriWeather/WeatherObsrInfo/GnrlWeather/getWeatherYearDayList"
params = {
    "serviceKey": authkey,
    "Page_Size": 20,
    "search_Year": 2020,
    "obsr_Spot_Code": "210852A001",
}

response = requests.get(url, params=params)
if response.status_code == 200:
    soup = BeautifulSoup(response.text, "xml")
    s = soup.find("item")

    parsed_dict = xmltodict.parse(str(s))
    test = pd.DataFrame(parsed_dict).T

test

,condens_Time,date,gr_Temp,hum,max_Temp,min_Temp,no,rain,soil_Temp,soil_Wt,stn_Code,stn_Name,sun_Qy,sun_Time,temp,widdir,wind
item,0,2020-01-01,0.8,36.8,6.7,-3,1,0,3.2,4.9,210852A001,강릉시 연곡면,11.6,531,1.4,259.8,3.8


In [57]:
def Get_AgriWeather_1(
        authkey = r"""b8m/px7AI1NzkWN/lcRj9uXgO56otbz7ZIBfXcplZtC7kmPv1qdNbWD9koAjSQp6t+eo2JJPcn+FiGNuklo98A==""",
        url = "http://apis.data.go.kr/1390802/AgriWeather/WeatherObsrInfo/GnrlWeather/getWeatherYearDayList",
        tag = "item",
        params = {
        "serviceKey": authkey,
        "Page_Size": 20,
        "search_Year": 2020,
        "obsr_Spot_Code": "210852A001"
        }
    ) -> None:

    response = requests.get(url, params=params)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "xml")
        s = soup.find(tag)

        parsed_dict = xmltodict.parse(str(s))
    
    return pd.DataFrame(parsed_dict).T